Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

# Deploy Azure ML Hardware Accelerated Models (HAM) to Databox Edge Machine - Using Azure CLI
This notebook will walk through creating and deploying a sample client that can test inferencing of an Azure ML HW Accelerated Model. The sample client image contains a script which uses the AzureML SDK to inference and IoT Hub/Device SDK to send messages to your IoT Hub.

This notebook will walk you through the following:

* [Install dependencies](#installation)
* [Configure your Azure CLI environment](#configuration)
* [Choose image to deploy](#choose-your-model)
* [Create Sample Client image](#sample-client)
* [Deploy modules using CLI](#deploy)
* [Receive IoT Hub Messages using Event Monitor](#iothub-messages)
* [Stop sample client module](#stop-module)
 
## Pre-requisites
* An IoT Edge Device (specifically, the Data Box Edge machine with Compute VM enabled)
    * Follow [steps 1 - 5a](https://docs.microsoft.com/en-us/azure/databox-online/data-box-edge-deploy-prep) to set up your Data Box Edge machine and enable the Linux Compute VM and an associated IoT Hub. This IoT Hub will deploy the AccelContainerImage to the Compute VM. **Note:** In order to use the Azure ML SDK to deploy your model to your IoT Hub, you must create your Data Box Edge resource in the same subscription as your Azure ML Workspace.
* An Azure ML ContainerImage or AccelContainerImage successfully created in your Workspace.
    * For an AccelContainerImage, you can follow [this notebook](https://github.com/Azure/MachineLearningNotebooks/blob/master/how-to-use-azureml/deployment/accelerated-models/accelerated-models-quickstart.ipynb) through Step 5.

<a id='installation'></a>
## Install Azure CLI, IoT extension, and Docker
The following code will:
- install Azure CLI and [Azure IoT CLI extension](https://github.com/Azure/azure-iot-cli-extension)
- install Docker
- sign into your Azure account using browser login

In [ ]:
%%bash
pip install azure-cli # Install Azure CLI
az extension add --name azure-cli-iot-ext # Add IoT CLI extension
pip install -U jupyter_console # Fix Jupyter dependencies overridden by Azure CLI installation
pip install docker # Install Docker
az login # Log into Azure account

<a id='configuration'></a>
## Configure default subscription and resource group

Next, we will set your default subscription and resource group for accessing your IoT Hub, assuming your IoT Hub is in the same subscription and resource group as your AzureML Workspace. You might need to customize these commands if your IoT Hub is in a different subscription or resource group.

In [ ]:
from azureml.core import Workspace
import os

ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')
os.environ["AZ_SUBSCRIPTION_ID"] = ws.subscription_id
os.environ["AZ_RESOURCE_GROUP"] = ws.resource_group

In [ ]:
%%bash
az account set --subscription $AZ_SUBSCRIPTION_ID
az configure --defaults group=$AZ_RESOURCE_GROUP

<a id="choose-your-model"></a>
## Choose the Azure ML HW Accelerated Model

List all Images in your Azure ML Workspace.

In [ ]:
from azureml.core.image import Image
for i in Image.list(workspace = ws):
    print('{}'.format(i.image_location))

#### Choose a converted image you'd like to deploy by copying the ACR path below. 
For example, "workspacename1234.azurecr.io/resnet50.1.brainwaveonnx-image:1".

In [ ]:
os.environ["FPGA_IMAGE_ACR"] = "COPY HERE"
os.environ["ACR_NAME"] = os.environ["FPGA_IMAGE_ACR"].split(".")[0]

print("You are choosing to use the Docker image {} in ACR {}.".format(os.environ["FPGA_IMAGE_ACR"], os.environ["ACR_NAME"]))

<a id='sample-client'></a>
## Create Sample Client Image

### Provide IoT Hub and IoT Edge Device information
When setting up your Compute VM for Databox Edge, you created an IoT Hub and Databox Edge automatically created an IoT Edge device registered to your Compute VM. In order to receive messages in your IoT Hub, copy in your IoT Hub Name and IoT Edge Device name to retrieve your connection string.

In [ ]:
%%bash
# List all iothubs associated with this resource group
az iot hub list --output table

#### Copy your IoT Hub name below.

In [ ]:
# Update IoT Hub and Edge Device information
os.environ["HUB_NAME"] = "COPY HERE"
# Databox Edge automatically uses your hub name appended with edge as your device name
os.environ["DEVICE_ID"] = "{}-edge".format(os.environ["HUB_NAME"])

os.environ["edgeAgent"] = "$edgeAgent"
os.environ["edgeHub"] = "$edgeHub"
os.environ["FILE_PATH"]="./deployment.json"

### Build sample client image and push to your ACR

The Docker image built from the /sample-client directory will work with the ResNet50 image created in Quickstart. To create a sample client that works with the Image created from transfer learning, update the line to below. The /sample-client uses ImageNet pictures for inferencing; the /sample-client-tl handles the response differently and uses images of cats and dogs. 

```
os.environ["CLIENT_APP_NAME"] = "sample-client-tl"
```

In [ ]:
# The default sample client works with the Docker image created from Quickstart
os.environ["CLIENT_APP_NAME"] = "sample-client"

In [ ]:
%%bash
az acr login --name $ACR_NAME
docker build -t $ACR_NAME.azurecr.io/$CLIENT_APP_NAME -f ./Dockerfile ./$CLIENT_APP_NAME
docker push $ACR_NAME.azurecr.io/$CLIENT_APP_NAME

<a id='deploy'></a>
## Deploy to Databox Edge Machine Using CLI
Here we will utilize the Azure CLI and a pre-defined deployment manifest template which will specify how to deploy your IoT Hub modules. 

When the Azure ML module first deploys, it will take a few minutes to flash the FPGA with the correct firmware for your model type. While this is happening, the sample client module will output that it is waiting for the flashing to finish.

Resources: 
- [az iot edge set-modules](https://docs.microsoft.com/en-us/azure/iot-edge/how-to-deploy-modules-cli)
- [az acr credential show](https://docs.microsoft.com/en-us/cli/azure/acr/credential?view=azure-cli-latest#az-acr-credential-show)
- [az iot hub device-identity show-connection-string](https://docs.microsoft.com/en-us/cli/azure/ext/azure-cli-iot-ext/iot/hub/device-identity?view=azure-cli-latest#ext-azure-cli-iot-ext-az-iot-hub-device-identity-show-connection-string)

#### Configure AzureML module
By default, the below is included in the generated deployment.json. You can modify it directly in that file if you want to, for example for a CPU image.
```
{
  "HostConfig": {
    "Privileged": true,
    "PortBindings": {
      "50051/tcp": [
        {
          "HostPort": "50051"
        }
      ]
    },
    "Devices": [
      {
        "PathOnHost": "/dev/catapult0",
        "PathInContainer": "/dev/catapult0"
      },
      {
        "PathOnHost": "/dev/catapult1",
        "PathInContainer": "/dev/catapult1"
      }
    ]
  }
}
```

#### Configure Sample Client module
By default, the below is included in the generated deployment.json. You can modify it directly in that file if you want to add other parameters.
```
{
  "Tty": true,
  "Cmd": [
    "--input-tensors",
    "Placeholder:0",
    "--output-tensors",
    "classifier/resnet_v1_50/predictions/Softmax:0",
    "--wait",
    "10"
  ]
}
```

If you just want to update the input and output tensor names for models other than resnet50 from Quickstart, run the cell below.

For example, the transfer learning notebook would use the same input tensor name and change the output tensor name.
```
os.environ["OUTPUT_TENSOR"] = "classifier_output/Softmax:0"
```

In [ ]:
os.environ["INPUT_TENSOR"] = "Placeholder:0"
os.environ["OUTPUT_TENSOR"] = "classifier/resnet_v1_50/predictions/Softmax:0"

The below code will replace all the values we've set previously in a newly generated file called deployment.json.

In [ ]:
%%bash
export ACR_PASSWORD=$(az acr credential show -n $ACR_NAME --query passwords[0].value | grep -oP '"\K[^"\047]+(?=["\047])')
export DEVICE_CONNECTION_STRING=$(az iot hub device-identity show-connection-string --device-id $DEVICE_ID --hub-name $HUB_NAME | grep -oP '(?<=\")HostName(.*)(?=\")')
export TEMPLATE_PATH="./deployment_template.json"
export HOST_STATUS="running"
export CLIENT_STATUS="running"

# Replace all environment variables in deployment.json
file=$(<$TEMPLATE_PATH)
echo "$file" | envsubst > $FILE_PATH

Now we use the Azure IoT CLI to set the deployment manifest for your Data Box Edge IoT Edge device.

In [ ]:
%%bash
# This command sets the deployment.json of your device, which includes the settings for both modules
az iot edge set-modules --device-id $DEVICE_ID --hub-name $HUB_NAME --content $FILE_PATH

<a id='iothub-messages'></a>
## Receive Predictions from IoT Hub
The code below uses the CLI to receive messages from IoT Hub. To use the IoT Service Client SDK, read more here: https://docs.microsoft.com/en-us/azure/iot-hub/iot-hub-devguide-messages-read-builtin

In [ ]:
%%bash
echo "Copy the following command into a bash window:"
echo ""
echo "az iot hub monitor-events --hub-name $HUB_NAME --device-id $DEVICE_ID --properties app"

<a id='stop-module'></a>
## Stop Sample Client Module
We can update the desired status of the IoT Hub modules. Here, we will update the deployment.json to specify that the sample client module should be "stopped". See the [README.md](./README.md) for more information about how to manipulate the sample client module.

In [ ]:
%%bash
export ACR_PASSWORD=$(az acr credential show -n $ACR_NAME --query passwords[0].value | grep -oP '"\K[^"\047]+(?=["\047])')
export DEVICE_CONNECTION_STRING=$(az iot hub device-identity show-connection-string --device-id $DEVICE_ID --hub-name $HUB_NAME | grep -oP '(?<=\")HostName(.*)(?=\")')
export TEMPLATE_PATH="./deployment_template.json"
export FILE_PATH="./deployment.json"
export HOST_STATUS="running"
export CLIENT_STATUS="stopped"

# Replace all environment variables in deployment.json
file=$(<$TEMPLATE_PATH)
echo "$file" | envsubst > $FILE_PATH

# This command sets the deployment.json of your device, which includes the settings for both modules
az iot edge set-modules --device-id $DEVICE_ID --hub-name $HUB_NAME --content $FILE_PATH